In [9]:
import sys
sys.path.append('../mydata/')
import scipy.io
import urllib.request
import dgl
import math
import numpy as np
import pandas as pd
from model import *
import torch
from data_loader import data_loader
from utils.data import load_data
from utils.pytorchtools import EarlyStopping
from utils.tools import evaluate_results_nc
import argparse
import time

def sp_to_spt(mat):
    coo = mat.tocoo()
    values = coo.data
    indices = np.vstack((coo.row, coo.col))

    i = torch.LongTensor(indices)
    v = torch.FloatTensor(values)
    shape = coo.shape

    return torch.sparse.FloatTensor(i, v, torch.Size(shape))


def mat2tensor(mat):
    if type(mat) is np.ndarray:
        return torch.from_numpy(mat).type(torch.FloatTensor)
    return sp_to_spt(mat)

def sampler(anchor, sample):
    assert anchor.size(0) == sample.size(0)
    num_nodes = anchor.size(0)
    device = anchor.device
    pos_mask = torch.eye(num_nodes, dtype=torch.float32, device=device)
    neg_mask = 1. - pos_mask
    return anchor, sample, pos_mask, neg_mask

def _similarity(h1: torch.Tensor, h2: torch.Tensor):
    h1 = F.normalize(h1)
    h2 = F.normalize(h2)
    return h1 @ h2.t()

def InfoNCE_loss(anchor, sample, pos_mask, neg_mask, tau=0.2):
        sim = _similarity(anchor, sample) / tau
        exp_sim = torch.exp(sim) * (pos_mask + neg_mask)
        log_prob = sim - torch.log(exp_sim.sum(dim=1, keepdim=True))
        loss = log_prob * pos_mask
        loss = loss.sum(dim=1) / pos_mask.sum(dim=1)
        return -loss.mean()
    
def TripletMargin_loss(anchor, sample, pos_mask, neg_mask=None, margin = 1.0, p = 2.0):
        loss_fn = torch.nn.TripletMarginLoss(margin=margin, p=p, reduction='none')
        num_anchors = anchor.size()[0]
        num_samples = sample.size()[0]

        # Key idea here:
        #  (1) Use all possible triples (will be num_anchors * num_positives * num_negatives triples in total)
        #  (2) Use PyTorch's TripletMarginLoss to compute the marginal loss for each triple
        #  (3) Since TripletMarginLoss accepts input tensors of shape (B, D), where B is the batch size,
        #        we have to manually construct all triples and flatten them as an input tensor in the
        #        shape of (num_triples, D).
        #  (4) We first compute loss for all triples (including those that are not anchor - positive - negative), which
        #        will be num_anchors * num_samples * num_samples triples, and then filter them with masks.

        # compute negative mask
        neg_mask = 1. - pos_mask if neg_mask is None else neg_mask

        anchor = torch.unsqueeze(anchor, dim=1).cpu()  # [N, 1, D]
        anchor = torch.unsqueeze(anchor, dim=1)  # [N, 1, 1, D]
        anchor = anchor.expand(-1, num_samples, num_samples, -1)  # [N, M, M, D]
        anchor = torch.flatten(anchor, end_dim=1)  # [N * M * M, D]

        pos_sample = torch.unsqueeze(sample, dim=0).cpu()  # [1, M, D]
        pos_sample = torch.unsqueeze(pos_sample, dim=2)  # [1, M, 1, D]
        pos_sample = pos_sample.expand(num_anchors, -1, num_samples, -1)  # [N, M, M, D]
        pos_sample = torch.flatten(pos_sample, end_dim=1)  # [N * M * M, D]

        neg_sample = torch.unsqueeze(sample, dim=0).cpu()  # [1, M, D]
        neg_sample = torch.unsqueeze(neg_sample, dim=0)  # [1, 1, M, D]
        neg_sample = neg_sample.expand(num_anchors, -1, num_samples, -1)  # [N, M, M, D]
        neg_sample = torch.flatten(neg_sample, end_dim=1)  # [N * M * M, D]

        loss = loss_fn(anchor.cuda(), pos_sample.cuda(), neg_sample.cuda())  # [N, M, M]
        loss = loss.view(num_anchors, num_samples, num_samples)

        pos_mask1 = torch.unsqueeze(pos_mask, dim=2).cpu()  # [N, M, 1]
        pos_mask1 = pos_mask1.expand(-1, -1, num_samples)  # [N, M, M]
        neg_mask1 = torch.unsqueeze(neg_mask, dim=1).cpu()  # [N, 1, M]
        neg_mask1 = neg_mask1.expand(-1, num_samples, -1)  # [N, M, M]

        pair_mask = pos_mask1.cuda() * neg_mask1.cuda()  # [N, M, M]
        num_pairs = pair_mask.sum()

        loss = loss * pair_mask
        loss = loss.sum()

        return loss / num_pairs

def build_heterograph(args):
    if args.device >= 0:
        device = torch.device("cuda:"+str(args.device))
    else:
        device = torch.device('cpu')
    
    print('loading data >>>>>>>>>>>>>>>>>>>>>')
    dataset = data_loader('../mydata/'+args.dataset)
    edge_dict = {}

    for i, meta_path in dataset.links['meta'].items():
        edge_dict[(str(meta_path[0]), str(meta_path[0]) + '_' + str(meta_path[1]), str(meta_path[1]))] = (torch.tensor(dataset.links['data'][i].tocoo().row - dataset.nodes['shift'][meta_path[0]]), torch.tensor(dataset.links['data'][i].tocoo().col - dataset.nodes['shift'][meta_path[1]]))

    node_count = {}
    for i, count in dataset.nodes['count'].items():
        print(i, node_count)
        node_count[str(i)] = count

    print('building dgl.heterograph >>>>>>>>>>>>>>>>>>>>>')
    G = dgl.heterograph(edge_dict, num_nodes_dict = node_count, device=device)
    """
    for ntype in G.ntypes:
        G.nodes[ntype].data['inp'] = dataset.nodes['attr'][ntype]
        # print(G.nodes['attr'][ntype].shape)
    """

    G.node_dict = {}
    G.edge_dict = {}
    for ntype in G.ntypes:
        G.node_dict[ntype] = len(G.node_dict)
    for etype in G.etypes:
        G.edge_dict[etype] = len(G.edge_dict)
        G.edges[etype].data['id'] = torch.ones(G.number_of_edges(etype), dtype=torch.long).to(device) * G.edge_dict[etype] 
#     print('G*****',G)
    return G,dataset

def run_model_DBLP(args, G, dataset, rep):
    if args.device >= 0:
        device = torch.device("cuda:"+str(args.device))
    else:
        device = torch.device('cpu')
    
    feats_type = args.feats_type
    features_list, adjM, labels, train_val_test_idx, dl = load_data(args.dataset)
    features_list = [mat2tensor(features).to(device) for features in features_list]

    if feats_type == 0:
        in_dims = [features.shape[1] for features in features_list]
    elif feats_type == 1 or feats_type == 5:
        save = 0 if feats_type == 1 else 2
        in_dims = []#[features_list[0].shape[1]] + [10] * (len(features_list) - 1)
        for i in range(0, len(features_list)):
            if i == save:
                in_dims.append(features_list[i].shape[1])
            else:
                in_dims.append(10)
                features_list[i] = torch.zeros((features_list[i].shape[0], 10)).to(device)
    elif feats_type == 2 or feats_type == 4:
        save = feats_type - 2
        in_dims = [features.shape[0] for features in features_list]
        for i in range(0, len(features_list)):
            if i == save:
                in_dims[i] = features_list[i].shape[1]
                continue
            dim = features_list[i].shape[0]
            indices = np.vstack((np.arange(dim), np.arange(dim)))
            indices = torch.LongTensor(indices)
            values = torch.FloatTensor(np.ones(dim))
            features_list[i] = torch.sparse.FloatTensor(indices, values, torch.Size([dim, dim])).to(device)
    elif feats_type == 3:
        in_dims = [features.shape[0] for features in features_list]
        for i in range(len(features_list)):
            dim = features_list[i].shape[0]
            indices = np.vstack((np.arange(dim), np.arange(dim)))
            indices = torch.LongTensor(indices)
            values = torch.FloatTensor(np.ones(dim))
            features_list[i] = torch.sparse.FloatTensor(indices, values, torch.Size([dim, dim])).to(device)

    ## 添加图特征
    for ntype in G.ntypes:
        G.nodes[ntype].data['inp'] = features_list[int(ntype)]#.to(device)
    labels = torch.LongTensor(labels).to(device)
    train_idx = train_val_test_idx['train_idx']
    train_idx = np.sort(train_idx)
    val_idx = train_val_test_idx['val_idx']
    val_idx = np.sort(val_idx)
    test_idx = train_val_test_idx['test_idx']
    test_idx = np.sort(test_idx)
    unlabel_idx = train_val_test_idx['unlabel_idx']
    unlabel_idx = np.sort(unlabel_idx)
    
    ## kk
    is_label = torch.ones(len(G.nodes['0'].data['inp'])).view(-1,1)
    is_label[unlabel_idx]=0
    for ntype in G.ntypes:
        if ntype == '0':
            G.nodes[ntype].data['is_label'] = is_label.to(device)
        else:
            G.nodes[ntype].data['is_label'] = torch.zeros(len(G.nodes[ntype].data['inp'])).view(-1,1).to(device)
    ## kk

    print(len(train_idx), len(test_idx), len(val_idx), len(unlabel_idx))
    model = HGT(G, n_inps=in_dims, n_hid=args.hidden_dim, n_out=labels.max().item()+1, n_layers=args.num_layers, n_heads=args.num_heads, unlabel_idx=unlabel_idx, use_norm = args.use_norm).to(device)

    optimizer = torch.optim.AdamW(model.parameters(), weight_decay=args.weight_decay)#, lr=args.lr, weight_decay=args.weight_decay)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, total_steps=args.schedule_step, max_lr = 1e-3, pct_start=0.05)

    early_stopping = EarlyStopping(patience=args.patience, verbose=True, save_path='../HGT/checkpoint/checkpoint_infonce_{}_{}l_{}th.pt'.format(args.dataset, args.num_layers, rep))
    train_step = 0
    for epoch in range(args.epoch):
        with torch.autograd.set_detect_anomaly(True):
            t_start = time.time()
            model.train()
            # , embeddings2 
            logits1, logits2, embeddings1, embeddings2 = model(G, '0')

            print('cross_entropy>>>>>>>> ',F.cross_entropy(logits1[train_idx], labels[train_idx].to(device)))
            print('mse_loss>>>>>>>> ',F.mse_loss(logits1[unlabel_idx],logits2[unlabel_idx]))
            
#             anchor, sample, pos_mask, neg_mask = sampler(anchor=logits1[unlabel_idx], sample=logits2[unlabel_idx])
#             loss_ul = InfoNCE_loss(anchor, sample, pos_mask, neg_mask, tau=0.2)
#             loss_ul = TripletMargin_loss(anchor, sample, pos_mask, neg_mask)
#             print('infonce_loss>>>>>>>> ',loss_ul)
            
            train_loss = (1-args.rate)*F.cross_entropy(logits1[train_idx], labels[train_idx].to(device)) + args.rate*F.mse_loss(logits1[unlabel_idx],logits2[unlabel_idx])
            pred = logits1.argmax(1)
            train_acc = (pred[train_idx] == labels[train_idx]).float().mean()
        

            optimizer.zero_grad()
            train_loss.backward()
            optimizer.step()
            train_step += 1
            scheduler.step(train_step)
            t_end = time.time()
        print('Epoch {:05d} | Train_Loss: {:.4f} | Train_acc: {:.4f} |Time: {:.4f}'.format(epoch, train_loss.item(), train_acc.item(), t_end-t_start))
        t_start = time.time()
        model.eval()
        with torch.no_grad():
            logits1, logits2, val_embeddings1, val_embeddings2 = model(G, '0')
            
            val_loss = F.cross_entropy(logits1[val_idx], labels[val_idx])
            pred = logits1.cpu().numpy().argmax(axis=1)
            pred2 = logits2.cpu().numpy().argmax(axis=1)
        val_acc   = (pred[val_idx] == labels[val_idx].cpu().numpy()).mean()
        test_acc  = (pred[test_idx] == labels[test_idx].cpu().numpy()).mean()
        test_pred_ = pred[test_idx]
        
        val_acc2   = (pred2[val_idx] == labels[val_idx].cpu().numpy()).mean()
        print('val_acc2>>>>>>',val_acc2)
        
        t_end = time.time()
        print('Epoch {:05d} | Val_Loss {:.4f} | Val_acc {:.4f} | Test_acc {:.4f} | Time(s) {:.4f}'.format(
            epoch, val_loss.item(), val_acc.item(), test_acc.item(), t_end - t_start))
        early_stopping(val_loss, model)
        if early_stopping.early_stop:
            print('Early stopping!')
            return model,test_idx,dataset,dl 
    return model,test_idx,dataset,dl
  
def inference(args, model, test_idx, dataset, G, rep, dl):
    model.load_state_dict(torch.load('../HGT/checkpoint/checkpoint_infonce_{}_{}l_{}th.pt'.format(args.dataset, args.num_layers, rep)))
    model.eval()
#     test_logits = []
    with torch.no_grad():
        logits, _, test_embeddings, _, = model(G, '0')
        test_logits = logits[test_idx]
        
        if args.dataset == 'IMDB1':
            pred = test_logits.cpu().numpy()
        else:
            pred = test_logits.cpu().numpy().argmax(axis=1)
        
        onehot = np.eye(dataset.labels_train['num_classes'], dtype=np.int32)
        pred = onehot[pred]
                
        eval_res = dataset.evaluate(pred)
        print(eval_res['micro-f1'], eval_res['macro-f1'])
        
        svm_macro_f1_list, svm_micro_f1_list, nmi_mean, nmi_std, ari_mean, ari_std = evaluate_results_nc(
                test_embeddings[test_idx].cpu().numpy(), dataset.labels_test['data'][dataset.labels_test['mask']].argmax(axis=1), num_classes=dataset.labels_test['num_classes'])

        return eval_res,svm_macro_f1_list, svm_micro_f1_list, nmi_mean, nmi_std, ari_mean, ari_std

In [ ]:

# if __name__ == '__main__':
ap = argparse.ArgumentParser(description='MRGNN testing for the DBLP dataset')
ap.add_argument('--feats-type', type=int, default=1,
                help='Type of the node features used. ' +
                        '0 - loaded features; ' +
                        '1 - only target node features (zero vec for others); ' +
                        '2 - only target node features (id vec for others); ' +
                        '3 - all id vec. Default is 2;' +
                    '4 - only term features (id vec for others);' + 
                    '5 - only term features (zero vec for others).')
ap.add_argument('--hidden_dim', type=int, default=64, help='Dimension of the node hidden state. Default is 64.')
ap.add_argument('--num_heads', type=int, default=8, help='Number of the attention heads. Default is 8.')
ap.add_argument('--epoch', type=int, default=300, help='Number of epochs.')
ap.add_argument('--patience', type=int, default=30, help='Patience.')
ap.add_argument('--repeat', type=int, default=5, help='Repeat the training and testing for N times. Default is 1.')
ap.add_argument('--num_layers', type=int, default=8) #5
ap.add_argument('--lr', type=float, default=1e-2) #5e-3
ap.add_argument('--dropout', type=float, default=0)
ap.add_argument('--weight-decay', type=float, default=1e-3) #1e-4
ap.add_argument('--slope', type=float, default=0.05)
ap.add_argument('--dataset', type=str, default='DBLP')  # Freebase
ap.add_argument('--edge-feats', type=int, default=64)
ap.add_argument('--device', type=int, default=0)
ap.add_argument('--schedule_step', type=int, default=300)
ap.add_argument('--use_norm', type=bool, default=True)
ap.add_argument('--rate', type=float, default=0.2)

args = ap.parse_args(args=[])
g, dataset = build_heterograph(args)


micro,macro = [],[]
svm_macro_f1_lists,svm_micro_f1_lists,nmi_mean_list,nmi_std_list,ari_mean_list,ari_std_list = [],[],[],[],[],[]
for r in range(args.repeat):
    print('running the '+str(r)+' times >>>>>>>>>>>>')
    model,test_idx,dataset,dl = run_model_DBLP(args, g, dataset, r)
    eval_res,svm_macro_f1_list, svm_micro_f1_list, nmi_mean, nmi_std, ari_mean, ari_std = inference(args, model, test_idx, dataset, g, r, dl)
    macro.append(eval_res['macro-f1'])
    micro.append(eval_res['micro-f1'])
    svm_macro_f1_lists.append(svm_macro_f1_list)
    svm_micro_f1_lists.append(svm_micro_f1_list)
    nmi_mean_list.append(nmi_mean)
    nmi_std_list.append(nmi_std)
    ari_mean_list.append(ari_mean)
    ari_std_list.append(ari_std)

    
m_macro = np.mean(macro)
std_macro = np.std(macro)
m_micro = np.mean(micro)
std_micro = np.std(micro)
sqrt_n = np.sqrt(len(micro))
print('------------------------------ final result ----------------------------------------')
print('final result macro_f1 >>>>>>>>>> ',m_macro,'+-',1.96*std_macro/sqrt_n)
print('final result micro_f1 >>>>>>>>>> ',m_micro,'+-',1.96*std_micro/sqrt_n)

# print out a summary of the evaluations
svm_macro_f1_lists = np.transpose(np.array(svm_macro_f1_lists), (1, 0, 2))
svm_micro_f1_lists = np.transpose(np.array(svm_micro_f1_lists), (1, 0, 2))
nmi_mean_list = np.array(nmi_mean_list)
nmi_std_list = np.array(nmi_std_list)
ari_mean_list = np.array(ari_mean_list)
ari_std_list = np.array(ari_std_list)
print('----------------------------------------------------------------')
print('SVM tests summary')
print('Macro-F1: ' + ', '.join(['{:.6f}~{:.6f} ({:.1f})'.format(
    macro_f1[:, 0].mean(), macro_f1[:, 1].mean(), train_size) for macro_f1, train_size in
    zip(svm_macro_f1_lists, [0.8, 0.6, 0.4, 0.2])]))
print('Micro-F1: ' + ', '.join(['{:.6f}~{:.6f} ({:.1f})'.format(
    micro_f1[:, 0].mean(), micro_f1[:, 1].mean(), train_size) for micro_f1, train_size in
    zip(svm_micro_f1_lists, [0.8, 0.6, 0.4, 0.2])]))
print('K-means tests summary')
print('NMI: {:.6f}~{:.6f}'.format(nmi_mean_list.mean(), nmi_std_list.mean()))
print('ARI: {:.6f}~{:.6f}'.format(ari_mean_list.mean(), ari_std_list.mean()))


loading data >>>>>>>>>>>>>>>>>>>>>
0 {}
1 {'0': 4057}
2 {'0': 4057, '1': 14328}
3 {'0': 4057, '1': 14328, '2': 7723}
building dgl.heterograph >>>>>>>>>>>>>>>>>>>>>
running the 0 times >>>>>>>>>>>>
975 2839 243 2839
cross_entropy>>>>>>>>  tensor(1.4909, device='cuda:0', grad_fn=<NllLossBackward>)
mse_loss>>>>>>>>  tensor(0.8932, device='cuda:0', grad_fn=<MseLossBackward>)
Epoch 00000 | Train_Loss: 1.9375 | Train_acc: 0.2564 |Time: 22.7640
val_acc2>>>>>> 0.25102880658436216
Epoch 00000 | Val_Loss 1.4370 | Val_acc 0.2716 | Test_acc 0.2888 | Time(s) 5.1331
Validation loss decreased (inf --> 1.436957).  Saving model ...
cross_entropy>>>>>>>>  tensor(1.5256, device='cuda:0', grad_fn=<NllLossBackward>)
mse_loss>>>>>>>>  tensor(0.8690, device='cuda:0', grad_fn=<MseLossBackward>)
Epoch 00001 | Train_Loss: 1.9601 | Train_acc: 0.2441 |Time: 22.1077
val_acc2>>>>>> 0.2551440329218107
Epoch 00001 | Val_Loss 1.4264 | Val_acc 0.2757 | Test_acc 0.2917 | Time(s) 5.0920
Validation loss decreased (1.43695

/opt/conda/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:154: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


Epoch 00000 | Train_Loss: 1.9529 | Train_acc: 0.2431 |Time: 21.8162
val_acc2>>>>>> 0.2716049382716049
Epoch 00000 | Val_Loss 1.6131 | Val_acc 0.2346 | Test_acc 0.2223 | Time(s) 5.1126
Validation loss decreased (inf --> 1.613110).  Saving model ...
cross_entropy>>>>>>>>  tensor(1.5831, device='cuda:0', grad_fn=<NllLossBackward>)
mse_loss>>>>>>>>  tensor(0.7348, device='cuda:0', grad_fn=<MseLossBackward>)
Epoch 00001 | Train_Loss: 1.9505 | Train_acc: 0.2441 |Time: 21.8424
val_acc2>>>>>> 0.2674897119341564
Epoch 00001 | Val_Loss 1.5964 | Val_acc 0.2510 | Test_acc 0.2216 | Time(s) 5.0247
Validation loss decreased (1.613110 --> 1.596351).  Saving model ...
cross_entropy>>>>>>>>  tensor(1.5714, device='cuda:0', grad_fn=<NllLossBackward>)
mse_loss>>>>>>>>  tensor(0.7446, device='cuda:0', grad_fn=<MseLossBackward>)
Epoch 00002 | Train_Loss: 1.9438 | Train_acc: 0.2554 |Time: 21.7347
val_acc2>>>>>> 0.2716049382716049
Epoch 00002 | Val_Loss 1.5707 | Val_acc 0.2305 | Test_acc 0.2311 | Time(s) 5.00

/opt/conda/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:154: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


Epoch 00000 | Train_Loss: 1.9483 | Train_acc: 0.2328 |Time: 21.6410
val_acc2>>>>>> 0.2716049382716049
Epoch 00000 | Val_Loss 1.4118 | Val_acc 0.2798 | Test_acc 0.2487 | Time(s) 4.9259
Validation loss decreased (inf --> 1.411824).  Saving model ...
cross_entropy>>>>>>>>  tensor(1.4745, device='cuda:0', grad_fn=<NllLossBackward>)
mse_loss>>>>>>>>  tensor(0.8604, device='cuda:0', grad_fn=<MseLossBackward>)
Epoch 00001 | Train_Loss: 1.9048 | Train_acc: 0.2687 |Time: 22.0344
val_acc2>>>>>> 0.2716049382716049
Epoch 00001 | Val_Loss 1.4059 | Val_acc 0.2840 | Test_acc 0.2628 | Time(s) 4.9972
Validation loss decreased (1.411824 --> 1.405873).  Saving model ...
cross_entropy>>>>>>>>  tensor(1.5127, device='cuda:0', grad_fn=<NllLossBackward>)
mse_loss>>>>>>>>  tensor(0.8383, device='cuda:0', grad_fn=<MseLossBackward>)
Epoch 00002 | Train_Loss: 1.9319 | Train_acc: 0.2462 |Time: 21.8325
val_acc2>>>>>> 0.2757201646090535
Epoch 00002 | Val_Loss 1.3978 | Val_acc 0.2593 | Test_acc 0.2814 | Time(s) 4.99

In [14]:

import pandas as pd
from data_loader import gen_file_for_evaluate
data = 'DBLP'

for i in range(5):
    df = pd.read_csv('../result/'+data+'_'+str(i)+'.txt', sep='\t', names=["node_id","node_name","node_type","label"])
    df[['label']].to_csv('../result1/'+data+'_'+str(i)+'.txt', sep='\t',index=False, header = False)

In [4]:
np.random.shuffle([1,2,3,4])

In [ ]:
1